### Step 1: Import Required Libraries and Initialize Earth Engine

 First, we need to import the necessary libraries and initialize Earth Engine.

In [1]:
import ee #For accessing Earth Engine
import geemap # Python library for accessing data in earth engine
import os # access our local computer 

### Step 2: Create a Map Object

We set up a map centered on Kenya with an appropriate zoom level.

In [3]:
# Create a map focused on Kenya
Map = geemap.Map(center=[0.0236, 37.9062], zoom=6)  # Kenya's coordinates

Map

Map(center=[0.0236, 37.9062], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

### Step 3: Load the Dataset

Here, we load the ESRI Global Land Use Land Cover (LULC) dataset.

In [4]:
# Load the ESRI Global LULC dataset
esri_lulc10 = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS")


### Step 4: Define a Legend

We define a dictionary with the names of land cover classes and their corresponding colors. This will be used both for visualization and for the legend.

In [5]:
# Define a dictionary for land cover class names and colors
legend_dict = {
    "Water": "#1A5BAB",
    "Trees": "#358221",
    "Flooded Vegetation": "#87D19E",
    "Crops": "#FFDB5C",
    "Built Area": "#ED022A",
    "Bare Ground": "#EDE9E4",
    "Snow/Ice": "#F2FAFF",
    "Clouds": "#C8C8C8",
    "Rangeland": "#C6AD8D"
}


### Step 5: Define the Kenya Border

We extract Kenya's border from a global administrative dataset for clipping the LULC images.

In [6]:
# Define the Kenya border using a global administrative dataset
kenyaBorder = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0") \
    .filter(ee.Filter.eq('ADM0_NAME', 'Kenya'))


### Step 6: Define a Function for Clipping and Remapping

This function clips the LULC images to Kenya's boundary and remaps the land cover class values.

In [7]:
# Function to clip images to Kenya's border and remap land cover classes
def clip_and_remap(image):
    return image.clip(kenyaBorder).remap(
        [1, 2, 4, 5, 7, 8, 9, 10, 11],  # Original class values
        [1, 2, 3, 4, 5, 6, 7, 8, 9]    # New class values for visualization
    )


### Step 7: Add LULC Layers for Multiple Years

We loop through each year, process the LULC data, and add it to the map.

In [9]:
# Loop through years and add LULC data to the map
years = list(range(2017, 2024))  # From 2017 to 2023
for year in years:
    start_date = f'{year}-01-01'  # Start of the year
    end_date = f'{year}-12-31'    # End of the year
    
    # Process the LULC data: filter, clip, remap, and mosaic
    processed_image = ee.ImageCollection(esri_lulc10.filterDate(start_date, end_date)) \
        .map(clip_and_remap) \
        .mosaic()
    
    # Add the processed image to the map
    Map.addLayer(processed_image, {"min": 1, "max": 9, "palette": list(legend_dict.values())}, f'{year} LULC 10m')
    
    
Map


Map(bottom=8491.0, center=[0.0236, 37.9062], controls=(WidgetControl(options=['position', 'transparent_bg'], w…

### Step 8: Add a Legend to the Map
We create and add a legend using the legend_dict.

In [10]:
# Function to add a legend to the map
def add_legend(map_obj, legend_dict):
    legend_title = "Land Cover Classes"  # Title for the legend
    map_obj.add_legend(legend_title=legend_title, legend_dict=legend_dict)

# Add the legend to the map
add_legend(Map, legend_dict)


### Step 9: Add Kenya's Border to the Map

To clearly show Kenya's boundary, we add it as a styled layer.

In [11]:
# Add Kenya's border to the map
Map.addLayer(kenyaBorder.style(color="black", width=2), {}, "Kenya Border")


In [12]:
Map

Map(bottom=8491.0, center=[0.0236, 37.9062], controls=(WidgetControl(options=['position', 'transparent_bg'], w…

In [15]:
# Process the LULC data for 2023
start_date = '2023-01-01'
end_date = '2023-12-31'

# Filter, clip, remap, and mosaic the LULC data for 2023
processed_image = ee.ImageCollection(esri_lulc10.filterDate(start_date, end_date)) \
    .map(clip_and_remap) \
    .mosaic()

# Specify the file path and download parameters
output_path = "LULC_2023_Kenya.tif"  # Local path where the file will be saved
region = kenyaBorder.geometry()  # Define the region to download

# Use geemap to download the image to the local directory
geemap.download_ee_image(
    image=processed_image,
    filename=output_path,
    region=region,
    scale=10,
    crs='EPSG:4326',  # Coordinate Reference System
)

print(f"LULC 2023 image downloaded to: {output_path}")

Consider adjusting `region`, `scale` and/or `dtype` to reduce the LULC_2023_Kenya.tif download size (raw: 9.26 GB).


LULC_2023_Kenya.tif: |          | 0.00/9.26G (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None


LULC 2023 image downloaded to: LULC_2023_Kenya.tif
